In [0]:

from pyspark.sql import functions as F


bronze_flights = spark.createDataFrame([
    ("AA101", "American", "JFK", "LAX", 10, 5, 0),
    ("DL202", "Delta", "LGA", "ORD", -5, 0, 0),
    ("UA303", "United", "ORD", "LAX", 20, 15, 0),
    ("AA101", "American", "JFK", "LAX", 10, 5, 0),  
    ("DL404", "Delta", "LAX", "JFK", 0, 0, 1)      
], ["flight_number", "airline", "origin", "dest", "arr_delay", "dep_delay", "cancelled"])

display(bronze_flights)


silver_flights = (
    bronze_flights
    .dropDuplicates()
    .withColumn("airline", F.upper(F.col("airline")))
    .withColumn("arr_delay", F.round(F.col("arr_delay")))
    .withColumn("dep_delay", F.round(F.col("dep_delay")))
)

display(silver_flights)



gold_flights = (
    silver_flights.groupBy("airline")
    .agg(
        F.count("flight_number").alias("total_flights"),
        F.round(F.avg("arr_delay"), 2).alias("avg_arr_delay"),
        F.round(F.avg("dep_delay"), 2).alias("avg_dep_delay"),
        F.sum("cancelled").alias("total_cancelled")
    )
)

display(gold_flights)


